In [1]:
DANDI26_I58     = "dandi://dandi/000026@draft/sub-I58"
LINC05_I58      = "dandi://linc/000005@draft/rawdata/sub-I58"
LINC05_I58_REG  = "dandi://linc/000005@draft/derivatives/registration-nitorch/sub-I58"
LINC05_I58_LOC  = "/Users/balbasty/localdata/linc/lincbrain/000005/derivatives/registration-nitorch/sub-I58"

HIPCT = {
    "hemi":      f"{DANDI26_I58}/ses-Hip-CT/micr/sub-I58_sample-01_chunk-01_hipCT.ome.zarr",
    "broca":     f"{DANDI26_I58}/ses-Hip-CT/micr/sub-I58_sample-01_chunk-02_hipCT.ome.zarr",
    "brainstem": f"{DANDI26_I58}/ses-Hip-CT/micr/sub-I58_sample-01_chunk-03_hipCT.ome.zarr",
    "ic2": {
        "101":   f"{LINC05_I58}/micr/sub-I58_sample-blockIC2_chunk-101_PC.ome.zarr",
        "201":   f"{LINC05_I58}/micr/sub-I58_sample-blockIC2_chunk-201_PC.ome.zarr",
        "401":   f"{LINC05_I58}/micr/sub-I58_sample-blockIC2_chunk-401_PC.ome.zarr",
    }
}

XFM = {
    "HIPCT_01_02":      f"{LINC05_I58_REG}/xfm/sub-I58_from-XPCT01_to-XPCT02_desc-final_xfm.lta",
    "HIPCT_01_03":      f"{LINC05_I58_REG}/xfm/sub-I58_from-XPCT01_to-XPCT03_desc-final_xfm.lta",
    "HIPCT_01_401":     f"{LINC05_I58_REG}/xfm/sub-I58_from-XPCT01_to-IC401_desc-finalfix_xfm.lta",
    "DWI_HIPCT_init":   f"{LINC05_I58_REG}/xfm/sub-I58_from-dwiHemi_to-XPCT01_desc-init_xfm.lta",
    "HIPCT_RAS":        f"{LINC05_I58_REG}/xfm/sub-I58_from-XPCT01_to-RAS_xfm.lta"
}

DWI = {
    "ic1": {
        "b0":   f"{LINC05_I58_REG}/dwi/sub-I58_sample-IC1_desc-b0_space-XPCT01_dwi.nii.zarr",
    },
    "hemi": {
        "b0":   f"{LINC05_I58_REG}/dwi/sub-I58_sample-hemi_desc-b0_space-XPCT01_dwi.nii.zarr",
        # "trk":  f"{LINC05_I58_REG}/dwi/sub-I58_desc-CSD_space-XPCT01_tractography.trk",
        "trk":  f"{LINC05_I58_LOC}/dwi/sub-I58_desc-CSD_space-XPCT01_tractography.trk",
    }
}

In [2]:
colormap_orient = """
vec3 colormapOrient(vec3 orient) {
  vec3 result;
  result.r = abs(orient[0]);
  result.g = abs(orient[1]);
  result.b = abs(orient[2]);
  return clamp(result, 0.0, 1.0);
}
"""

# Runing this in the skeleton shader uses orientation to color tracts
orient_shader = colormap_orient + """
#uicontrol bool orient_color checkbox(default=true)
void main() {
  if (orient_color)
    emitRGB(colormapOrient(orientation));
  else
    emitDefault();
}
"""

def rotate_orient_shader(mat):
    return (colormap_orient +
"""
#uicontrol bool orient_color checkbox(default=true)
void main() {
"""
+
f"""
    mat3 mat = mat3(
      {mat[0, 0]}, {mat[1, 0]}, {mat[2, 0]},
      {mat[0, 1]}, {mat[1, 1]}, {mat[2, 1]},
      {mat[0, 2]}, {mat[1, 2]}, {mat[2, 2]}
    );
    vec3 orient_rot = mat * orientation;
  if (orient_color)
    emitRGB(colormapOrient(orient_rot));
  else
    emitDefault();
"""
+
"""
}
"""
)


ic_shader = """
#uicontrol invlerp normalized
void main() {
  vec4 rgba;
  rgba.r = normalized();
  rgba.a = normalized();
  emitRGBA(rgba);
}
"""

In [3]:
# from ngtools.scene import Scene
from ngtools.local.viewer import LocalNeuroglancer

# import logging
# logging.basicConfig()
# logging.getLogger().setLevel(0)

viewer = LocalNeuroglancer()
print('viewer:      ', viewer.get_viewer_url())
print('fileserver:  ', viewer.get_fileserver_url())

viewer.load({"HiP-CT (hemi)":               "zarr://" + HIPCT["hemi"]})
viewer.load({"HiP-CT (broca)":              "zarr://" + HIPCT["broca"]},      transform=XFM["HIPCT_01_02"])
viewer.load({"HiP-CT (brainstem)":          "zarr://" + HIPCT["brainstem"]},  transform=XFM["HIPCT_01_03"])
viewer.load({"HiP-CT (internal capsule)":   "zarr://" + HIPCT["ic2"]["401"]}, transform=XFM["HIPCT_01_401"])
viewer.load({"dMRI (hemi)":                 "zarr://" + DWI["hemi"]["b0"]})
viewer.load({"dMRI (internal capsule)":     "zarr://" + DWI["ic1"]["b0"]})
viewer.load({"Tractography (hemi)":         "trk://"  + DWI["hemi"]["trk"]})
viewer.transform(XFM["HIPCT_RAS"], inv=True)
viewer.space("radio")

# rotate tract colors
with viewer.scene() as scene:
    hipct_ras = scene.layers["HiP-CT (hemi)"].source[0].transform.matrix
    scene.layers["Tractography (hemi)"].skeleton_shader = rotate_orient_shader(hipct_ras[:3, :3])
    scene.showSlices = False


viewer:       http://1.0.0.127.in-addr.arpa:59839/v/1/
fileserver:   http://0.0.0.0:59838/
Loaded: {'type': 'image', 'source': [{'url': 'zarr2://https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61/'}], 'shaderControls': {'normalized': {'range': [23005.0, 27807.0], 'window': [3412.0, 57812.0]}}, 'name': 'HiP-CT (hemi)'}
Loaded: {'type': 'image', 'source': [{'url': 'zarr2://https://dandiarchive.s3.amazonaws.com/zarr/a2861b6d-074a-46cc-a221-b6e9a75dc6be/'}], 'shaderControls': {'normalized': {'range': [13225.0, 17572.0], 'window': [9912.0, 48994.0]}}, 'name': 'HiP-CT (broca)'}
Loaded: {'type': 'image', 'source': [{'url': 'zarr2://https://dandiarchive.s3.amazonaws.com/zarr/1904d2d9-ed31-4cf5-b723-eedff28d2c7c/'}], 'shaderControls': {'normalized': {'range': [7933.0, 10301.0], 'window': [3804.0, 61110.0]}}, 'name': 'HiP-CT (brainstem)'}
Loaded: {'type': 'image', 'source': [{'url': 'zarr2://https://neuroglancer.lincbrain.org/zarr/6d25507c-ae9d-4f61-9a92-8644783adfd9

In [4]:
import webbrowser

webbrowser.open(viewer.get_viewer_url())
# webbrowser.open(viewer.state(url=True))

True